## Regresssion analysis to determine memorable characteristics that have significant inlfuence on ratings

#### Author: Lia Chin-Purcell
#### Date: 5/4/2022

In [1]:
# import our libraries
import io
import pandas as pd

import numpy as np 
import statsmodels.api as sm

## Vizualise
%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt


In [2]:
matplotlib.__version__

'3.5.1'

## Read in the chocolate bar rating dataset

We use a publicly [availiable chocolate bar rating dataset](http://flavorsofcacao.com/chocolate_database.html)

In [3]:
df_choc = pd.read_csv('../data/chocolate.csv')

In [4]:
df_choc.head(5)

,REF,Company (Manufacturer),Company Location,Review Date,Country of Bean Origin,Specific Bean Origin or Bar Name,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating
0,1205,Habitual,Canada,2014,Blend,one hundred,100%,NaN,"unrefined, bitter, earthy",2.00
1,701,Haigh,Australia,2011,Blend,South America and Africa,70%,NaN,"vanilla, chocolate milk",3.00
2,1113,Hotel Chocolat,U.K.,2013,St. Lucia,"Island Growers, 2012, 120hr c., batch 13080",100%,NaN,"pastey, bitter, unfixable",1.75
3,296,Hotel Chocolat (Coppeneur),U.K.,2008,Uganda,Uganda,80%,NaN,"charred, espresso",2.50
4,552,Hotel Chocolat (Coppeneur),U.K.,2010,Ecuador,Ecuador,70%,NaN,"spicy, sour, burning",2.75


# Most memorable characteristic regression

Here, we use a regresssion analysis to determine memorable characteristics that have significant inlfuence on ratings. For example, does 'bitter' have a positive or negative relationship with rating? What about 'creamy'? Which memorable characteristics have the most statistically significant effect, and which have the largest effect?

Memorable characteristics of a bar are terms generally relating to anything from texture, flavor, overall opinion, etc

First, we need to parse the memorable characteristics string as an array

In [5]:
dif_chars = []
most_mem = []

# new field for array version
df_choc['most_memorable'] = df_choc['Most Memorable Characteristics']


for index, row in df_choc.iterrows():
    mem_chars = row['Most Memorable Characteristics']
    # turn into array
    mem_chars = mem_chars.split(',')
    # strip
    mem_chars = [cha.strip() for cha in mem_chars]
    
    df_choc.at[index, 'most_memorable'] = mem_chars
    # print(mem_chars)
df_choc.head(5)


,REF,Company (Manufacturer),Company Location,Review Date,Country of Bean Origin,Specific Bean Origin or Bar Name,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating,most_memorable
0,1205,Habitual,Canada,2014,Blend,one hundred,100%,NaN,"unrefined, bitter, earthy",2.00,"[unrefined, bitter, earthy]"
1,701,Haigh,Australia,2011,Blend,South America and Africa,70%,NaN,"vanilla, chocolate milk",3.00,"[vanilla, chocolate milk]"
2,1113,Hotel Chocolat,U.K.,2013,St. Lucia,"Island Growers, 2012, 120hr c., batch 13080",100%,NaN,"pastey, bitter, unfixable",1.75,"[pastey, bitter, unfixable]"
3,296,Hotel Chocolat (Coppeneur),U.K.,2008,Uganda,Uganda,80%,NaN,"charred, espresso",2.50,"[charred, espresso]"
4,552,Hotel Chocolat (Coppeneur),U.K.,2010,Ecuador,Ecuador,70%,NaN,"spicy, sour, burning",2.75,"[spicy, sour, burning]"


In [6]:
# get dummies from array of memorable characteristics, so each feature is a dummy for that characteristic
df_tree = pd.get_dummies(df_choc['most_memorable'].explode()).groupby(level=0).sum()
df_tree.head(5)

,,Fruity,Rich cocoa,Roasty,XL nibs,accessible,acidic,alcohol,almond,almond butter,...,well developed,wet,wheat,wild berries,wild berry,wine,woodsy,woody,yellow fruit,yogurt
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# set up the regression

# outcome variable
y = df_choc['Rating']

# memorable characteristic dummies
X = df_tree

# add a constant
X = sm.add_constant(X)

# declare and fit the model
model = sm.OLS(y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Rating   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.596
Method:                 Least Squares   F-statistic:                     4.261
Date:                Thu, 12 May 2022   Prob (F-statistic):           3.12e-87
Time:                        00:16:09   Log-Likelihood:                 272.49
No. Observations:                1560   AIC:                             867.0
Df Residuals:                     854   BIC:                             4646.
Df Model:                         705                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                              3.2853      0.052     63.639      0.000       3.184       3.387
                                   0.0846      0.165      0.513      0.608      -0.239       0.408
Fruity                             0.1074      0.140      0.768      0.442      -0.167       0.382
Rich cocoa                        -0.0773      0.141     -0.548      0.584      -0.354       0.199
Roasty                             0.1074      0.140      0.768      0.442      -0.167       0.382
XL nibs                           -0.1515      0.300     -0.506      0.613      -0.740       0.437
accessible                         0.2152      0.202      1.066      0.287      -0.181       0.611
acidic                            -0.1206      0.071     -1.707      0.088      -0.259       0.018
alcohol                            0.0037      0.207      0.018      0.986      -0.402       0.409
almond                             0.0485      0.404      0.120      0.904      -0.745       0.842
almond butter                      0.1323      0.278      0.476      0.634      -0.413       0.678
almost burnt                      -0.7187      0.279     -2.577      0.010      -1.266      -0.171
anise                              0.0830      0.146      0.569      0.569      -0.203       0.369
appealing smoke                    0.4650      0.281      1.654      0.099      -0.087       1.017
apple                              0.3879      0.393      0.987      0.324      -0.383       1.159
artificial                        -0.3003      0.289     -1.039      0.299      -0.868       0.267
ashey                             -0.3532      0.149     -2.373      0.018      -0.645      -0.061
assertive                          0.2109      0.289      0.730      0.466      -0.356       0.778
astingent                         -0.4901      0.285     -1.722      0.085      -1.049       0.068
astringent                        -0.0555      0.063     -0.875      0.382      -0.180       0.069
atypical                          -0.2484      0.282     -0.882      0.378      -0.801       0.304
baked                             -0.3083      0.280     -1.102      0.271      -0.858       0.241
baked bread                        0.2147      0.279      0.768      0.442      -0.334       0.763
balanced                           0.2344      0.089      2.629      0.009       0.059       0.409
banana                             0.1415      0.075      1.893      0.059      -0.005       0.288
banana raspberry                   0.4291      0.321      1.336      0.182      -0.201       1.060
base cocoa                         0.0046      0.201      0.023      0.982      -0.389       0.399
base cocoa w/ palm notes           0.2147      0.279      0.768      0.442      -0.334       0.763
basic         

### Thats a lot of features! which ones are statistically significant?

## Positive effect memorable characteristics

Lets go through our model and filter on p-value less than 0.05, and positive coeficients. This indicates the characteristic had a statistically significant effect on rating, and that effect was *positive*. In other words, if the bar had this characteristic, the rating is likley to be higher - specifically `coef` points higher.

In [7]:
# create dataframe to be able to sort on p values with model information and parameters

ps = []
coefs = []
most_memorable = []

for i, var in enumerate(X.columns.tolist()):
    ps.append(model.pvalues[var])
    coefs.append(model.params[var])
    most_memorable.append(var)


In [8]:
# dataframe for output

df_params = pd.DataFrame()
df_params['p-value'] = ps
df_params['coef'] = coefs
df_params['name'] = most_memorable

In [9]:
df_params_sig = df_params[(df_params['p-value'] <= 0.05) & (df_params['coef'] > 0)]
# constant is not relevant
df_params_sig = df_params_sig[df_params_sig['name'] != 'const']

In [24]:
df_params_sig.head(10)

,p-value,coef,name
23,8.718429e-03,0.234444,balanced
38,2.504779e-03,0.868794,black current
43,5.182544e-04,0.420081,blackberry
47,1.547497e-02,0.970770,blueberries
59,3.637186e-02,0.206921,bright fruit
70,1.872272e-02,0.714407,burnt brownie
97,2.626219e-02,0.464276,cardamom
98,1.070548e-02,0.714730,cardamon
120,1.070548e-02,0.714730,chocolate and grapes
128,4.846590e-02,0.365115,cinamon


### save as a table

In [10]:
df_params_sig.to_csv('../tables/good_review_memorable_characteristics.csv')

## Negative effect memorable characteristics

Lets do the same for negative characteristics. Turns out there are more statistically significant terms for negative characteristics than positive ones, so lets make our significance threshold lower, 0.001.

In [11]:
df_params_sig = df_params[(df_params['p-value'] <= 0.001) & (df_params['coef'] < 0)]

In [12]:
df_params_sig.head(10)

,p-value,coef,name
35,2.097273e-27,-0.598600,bitter
79,3.158706e-04,-0.611484,burnt rubber
81,2.250781e-04,-1.035270,burnt up front
109,2.912281e-06,-0.463721,chemical
110,2.781201e-04,-0.733152,chemical off
176,3.395652e-04,-0.262095,dirty
188,1.644918e-08,-1.586933,dominate off note
198,1.567257e-05,-0.145713,earthy
233,7.896309e-05,-0.649563,fuel
259,1.813871e-05,-0.430084,harsh


### save as a table

In [13]:
df_params_sig.to_csv('../tables/bad_review_memorable_characteristics.csv')